# Neural Network from Scratch

Creating a neural network from scratch involves understanding the fundamental components and their interactions.

We shall study here designing sequential model using 
- `Neuron`
- `Layer`
- `Sequential`

we shall also use:
- `Sigmoid` (activation)
- `MeanSquaredError`(loss)
- `SGD` (Optimizer)

<br/>

___

## `Neuron` Class

we already developed `Neuron` class in [Neuron.py](../../lib/current/src/neuron/Neuron.py).

but to understand easily, we shall use a simpler version here

In [16]:
import numpy as np

class Neuron:
  output = None
  input = None
  delta = None

  def __init__(self, num_inputs, activation):
    self.weights = np.repeat(0.5, num_inputs)
    self.bias = 0.2
    self.activation = activation

  def forward(self, inputs):
    self.input = inputs
    z = np.dot(inputs, self.weights) + self.bias
    self.output = self.activation.apply(z)
    return self.output

  def backward(self, error, learning_rate):
    derivative = self.activation.derivative(self.output)
    self.delta = error * derivative
    self.weights -= learning_rate * self.delta * self.input
    self.bias -= learning_rate * self.delta
    return np.dot(self.delta, self.weights)


our `Neuron` need an `activation` object in consttructor for initialization.

in `forward`, it applies activation on weighted sum

in `backward` it applies SGD, and returns error gradient for previous neurons

<br/>

___

## `Layer` Class

`Layer` of `Neuron`s

In [17]:
class Layer:
  def __init__(self, num_neurons, num_inputs, activation):
    self.neurons = [Neuron(num_inputs, activation) for _ in range(num_neurons)]

  def forward(self, inputs):
    outputs = np.array([neuron.forward(inputs) for neuron in self.neurons])
    return outputs

  def backward(self, errors, learning_rate):
    next_errors = np.zeros(len(self.neurons[0].weights))
    for i, neuron in enumerate(self.neurons):
      next_errors += neuron.backward(errors[i], learning_rate)
    return next_errors


`__init__` (constructor) initialize all `Neuron`s

`forward` predicts outputs of all neurons

`backward` performs backward for every `Neuron` and calculate sum for previous layers

<br/>

___

## `Sequential` Class

make an ANN from `Layer`s

In [18]:
class Sequential:
  def __init__(self, layers=None):
    self.layers = layers or []

  def add(self, layer):
    self.layers.append(layer)

  def forward(self, inputs):
    for layer in self.layers:
      inputs = layer.forward(inputs)
    return inputs

  def backward(self, error, learning_rate):
    for layer in reversed(self.layers):
      error = layer.backward(error, learning_rate)

  def train(self, X, y, loss, epochs, learning_rate):
    for _ in range(epochs):
      for i in range(len(X)):
        output = self.forward(X[i])
        error = loss.calculate(y[i], output)
        self.backward(error, learning_rate)


`__init__` initializes layers. `add` appends

`forward` iteratively pass inputs of previous layer to next layer and returns output of last layer

`backward` performs reverse iteration on layers and pass gradient to previous layers

`train` apply epochs, for every epoch, output is chained from input layer to last layer, then output of last layer is fed to loss function. once error is calculated, loss is chained back for GD

<br/>

___

## Activation and Loss classes

In [19]:
class Activation:
  def apply(self, x): return 1 / (1 + np.exp(-x))
  def derivative(self, x): return x * (1 - x)

class Loss:
  def calculate(self, y, y_pred): return y_pred - y

## Usage

In [20]:
model = Sequential([
  Layer(3, 3, Activation()),
  Layer(1, 3, Activation()),
])

In [21]:
from dataset import X, Y

In [22]:
model.train(
  X, Y,
  loss=Loss(),
  epochs=20000,
  learning_rate=0.1
)

In [23]:
# testing
for i in range(len(X)):
  print('predicted =', round(model.forward(X[i])[0], 1), 'actual =', Y[i])

predicted = 1.0 actual = 1
predicted = 0.0 actual = 0
predicted = 1.0 actual = 1
predicted = 0.0 actual = 0
predicted = 0.0 actual = 0
predicted = 0.0 actual = 0
predicted = 1.0 actual = 1
predicted = 1.0 actual = 1
predicted = 1.0 actual = 1
predicted = 0.0 actual = 0
